#**A .Introduction:**
***A.1. Description & Discussion of the Background***

An entrepreneur wants to start a new Multi Cuisine Indian restaurant in CHICAGO. Since he believes that Chicago is one of the most densely populated Indian immigrant states in US. So
He approaches a DataScientist for a suggestion where to start his business. (source from wikipedia)

![alt text](https://github.com/kuruvasatya/Coursera_Capstone/blob/master/stats.png)

Since being a Data Scientist we need to solve his problem and recommend the best neighborhood of chicago to start his business.

**A.2. Problem :**
1. Finding all the Neighborhoods of Chicago.
1. Finding all the Neighborhoods with INDIAN Restaurants.
1. Finding all the Neighborhood with-out single INDIAN Restaurants.
1. Finding the best place and most visited INDIAN Restaurant.
1. Suggest best Neighborhood for starting business.


# ***1. Importing Necessary Libraries***

In [0]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
import folium

#***2. FourSquare Credentials***

In [23]:
CLIENT_ID = 'XKZVCIL3SUAANTTB2NYBEUAMMHBA43JF5J3ZIVVEFTA005BC' # your Foursquare ID
CLIENT_SECRET = 'FURVDY4HEITGFWS2TLVQSQWOXB5KZGT2FB44IDC0VONSF22E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XKZVCIL3SUAANTTB2NYBEUAMMHBA43JF5J3ZIVVEFTA005BC
CLIENT_SECRET:FURVDY4HEITGFWS2TLVQSQWOXB5KZGT2FB44IDC0VONSF22E


#***3.Getting Neighborhoods of Chicago from wikipedia***

In [16]:
# Reading Chicago city Neighborhoods from wikipedia
chicago_Neighborhoods = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago')[0]
chicago_Neighborhoods

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square
...,...,...
241,Wildwood,Forest Glen
242,Woodlawn,Woodlawn
243,Wrightwood,Ashburn
244,Wrightwood Neighbors,Lincoln Park


In [0]:
#retreiving the Neighborhood names for further analysis
chicago_Neighborhoods_names = list(chicago_Neighborhoods['Neighborhood'])

In [5]:
data = pd.DataFrame(columns=['Name','Latitude','Longitude'])
data

,Name,Latitude,Longitude


#***4.Assigning Latitude and Longitude for each Neighborhoods***

In [0]:
#Function that converts address into Latitude and Longitude
def geo_location(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    if(location == None):
      return [None,None]
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude
#filling the table
for place in chicago_Neighborhoods_names:
  address = place + ', Chicago'
  latitude,longitude = geo_location(address)
  if(latitude == None):
    continue
  data = data.append({'Name': place, 'Latitude': latitude,'Longitude': longitude},ignore_index=True)

data.to_csv('Chicago.csv')

In [6]:
# Reading the CSV file instead of computing multiple times
data = pd.read_csv('Chicago.csv')
#removing index
data = data.iloc[:,1:]
data

,Name,Latitude,Longitude
0,Albany Park,41.971937,-87.716174
1,Altgeld Gardens,41.654864,-87.600446
2,Andersonville,41.977139,-87.669273
3,Archer Heights,41.811422,-87.726165
4,Armour Square,41.840033,-87.633107
...,...,...,...
226,Wicker Park,41.907802,-87.678155
227,Wildwood,42.004691,-87.775924
228,Woodlawn,41.779479,-87.599493
229,Wrightwood,41.748932,-87.703575


In [9]:
#Locating all the Neighborhood on Map
#first problem to find all the Neighbourhoos is solved
map = folium.Map(location= [41.8781,-87.6298])

for name,lat,lng in zip(data.Name,data.Latitude,data.Longitude):
  folium.Marker(location=[lat,lng],popup='<strong> '+name+'</strong>').add_to(map)

map

#**5. Creating function that retreives all INDIAN resturants in Neighborhoods**

In [0]:
#fuction that retreives venues using Foursquare 
#Gives all the venues with in given Radius
def get_venues(lat,lng):
    #set variables
    radius=1000
    LIMIT=100

    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


#***6. Creating function that gets Ratings of all the INDIAN resturants***

In [0]:
#TO find the ratings of the venue

def get_venue_details(venue_id):    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

#***7. Finding all the INDIAN resturants in Neighborhood by calling get_venue function***

In [0]:
# Finding all Neighborhoods having INDIAN resturants

col_names = ['Neighborhood','Count']
# To keep count of how many resturants in each neighboorhood
counts = pd.DataFrame(columns=col_names)

#Actual details of the resturants
column_names=['Neighborhood', 'Id','Name_of_resturant']
indian_resturants=pd.DataFrame(columns=column_names)

for row in data.values.tolist():
  #for each neighborhood
  Neighborhood, Latitude, Longitude = row
  venues = get_venues(Latitude,Longitude)
  #selecting only indian resturants
  indian_resturants_details=venues[venues['Category']=='Indian Restaurant'] 
  #adding no Indian resturants found in each neighborhood
  counts = counts.append({'Neighborhood':Neighborhood,'Count':len(indian_resturants_details)},ignore_index=True)

  for resturant_details in indian_resturants_details.values.tolist():
      id , name , category= resturant_details
      indian_resturants = indian_resturants.append({'Neighborhood': Neighborhood, 
                                                      'Id': id,
                                                      'Name_of_resturant' : name}, ignore_index=True)


In [60]:
#finding Number of Restaurants in Each Neighborhood
counts

,Neighborhood,Count
0,Albany Park,0
1,Altgeld Gardens,0
2,Andersonville,0
3,Archer Heights,0
4,Armour Square,1
...,...,...
231,Albany Park,0
232,Altgeld Gardens,0
233,Andersonville,0
234,Archer Heights,0


In [0]:
#saving the file instead of computing multiple times
counts.to_csv('Counts.csv')

In [30]:
# List of INDIAN Resturants and their Location
indian_resturants

,Neighborhood,Id,Name_of_resturant
0,Armour Square,57205dd2498e101ee1162dbf,Oberoi's Indian Food
1,Avondale,5d2a84fc7cfbfe00235364df,Superkhana International
2,Avondale Gardens,5d2a84fc7cfbfe00235364df,Superkhana International
3,Belmont Central,562987d2498e027ba6c80810,Khyber Pass
4,Belmont Gardens,562987d2498e027ba6c80810,Khyber Pass
5,Belmont Terrace,562987d2498e027ba6c80810,Khyber Pass
6,Bowmanville,4b7ca417f964a5200b9f2fe3,Paprika
7,Bronzeville,57205dd2498e101ee1162dbf,Oberoi's Indian Food
8,East Hyde Park,4a592bdaf964a520dfb81fe3,Rajun Cajun
9,Edgewater,4b3c09dbf964a520cf8025e3,The Little India Restaurant


In [0]:
#saving the file 
indian_resturants.to_csv('Indian_Resturants.csv')

In [32]:
indian_resturants.groupby('Neighborhood')['Id'].count()

Neighborhood
Armour Square                 2
Avondale                      1
Avondale Gardens              1
Belmont Central               1
Belmont Gardens               1
Belmont Terrace               1
Bowmanville                   1
Bronzeville                   1
East Hyde Park                1
Edgewater                     1
Edgewater Beach               1
Edgewater Glen                1
Forest Glen                   1
Graceland West                1
Heart of Chicago              2
Hyde Park                     1
Illinois Medical District     1
Jackowo                       1
K-Town                        1
Lincoln Park                  1
Lincoln Square                1
Logan Square                  1
Margate Park                  1
Mayfair                       1
Merchant Park                 1
Noble Square                  1
Old Town                      1
Polish Village                1
Ravenswood Gardens            2
Ravenswood Manor              1
Rogers Park                

# ***It is evident that West Ridge is the Best Location where we find Larger Number of Indian Resturants***

In [34]:
# Resturants in West Ridge
West_Ridge = indian_resturants[indian_resturants['Neighborhood']=='West Ridge']
West_Ridge

,Neighborhood,Id,Name_of_resturant
38,West Ridge,4aa1e192f964a520aa4120e3,Udupi Palace
39,West Ridge,4bd492ce7b1876b09c518986,Tiffin Indian Kitchen
40,West Ridge,4b37dedaf964a520c64725e3,Tahoora
41,West Ridge,5062656be4b0b51b64824638,Clifton Grill
42,West Ridge,4b63a9f5f964a520ad8a2ae3,Khan B.B.Q.
43,West Ridge,4abc1f19f964a520778620e3,Hema's Kitchen
44,West Ridge,4f5e3fbae4b026f4c7e564ca,Pak Sweets
45,West Ridge,4a533e3ef964a52025b21fe3,Mysore Woodlands
46,West Ridge,4a426e47f964a520daa51fe3,Uru-Swati
47,West Ridge,4b09dd0cf964a520d71e23e3,Sukhadia's


In [0]:
#saving all the resturants details in west ridge
West_Ridge.to_csv('Resturants_in_West_Ridge.csv')

In [0]:
#getting Ratings of all the resturants in the WEST_RIDGE
column_names=['Neighborhood', 'ID','Name','Likes','Rating']
West_Ridge_resturants_Ratings=pd.DataFrame(columns=column_names)

for row in West_Ridge.values.tolist():
    Neighborhood,Id,Name_of_Resturant= row
    try:
        venue_details=get_venue_details(Id)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    West_Ridge_resturants_Ratings = West_Ridge_resturants_Ratings.append({
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)

In [39]:
#all the Ratings of the Resturants in West Ridge
West_Ridge_resturants_Ratings.sort_values(by='Rating',ascending=False)

,Neighborhood,ID,Name,Likes,Rating,Tips
2,West Ridge,4b37dedaf964a520c64725e3,Tahoora,49,7.8,25.0
3,West Ridge,5062656be4b0b51b64824638,Clifton Grill,6,7.8,2.0
0,West Ridge,4aa1e192f964a520aa4120e3,Udupi Palace,36,7.7,23.0
1,West Ridge,4bd492ce7b1876b09c518986,Tiffin Indian Kitchen,45,7.6,27.0
4,West Ridge,4b63a9f5f964a520ad8a2ae3,Khan B.B.Q.,36,7.5,19.0
6,West Ridge,4f5e3fbae4b026f4c7e564ca,Pak Sweets,5,7.3,4.0
5,West Ridge,4abc1f19f964a520778620e3,Hema's Kitchen,41,7.2,23.0
10,West Ridge,4c25698b905a0f4739946160,Delhi Darbar Kabab House,7,7.2,8.0
9,West Ridge,4b09dd0cf964a520d71e23e3,Sukhadia's,28,6.9,15.0
8,West Ridge,4a426e47f964a520daa51fe3,Uru-Swati,43,6.9,31.0


# ***It is evident that Tahoora Resturant in West_Ridge is the best place to dine as it having a rating of 7.8 and 49 likes***

In [0]:
#saving the ratings of each resturants
West_Ridge_resturants_Ratings.to_csv('Ratings.csv')

In [0]:
#finding neighborhood areas with no resturants
Neighborhood_with_no_resturants = counts[counts['Count']==0].reset_index(drop=True)

In [0]:
#finding neighborhoof area with resturants
Neighborhood_areas_with_Resturants = counts[counts['Count']>0].reset_index(drop=True)

In [63]:
Neighborhood_areas_with_Resturants.sort_values(by='Count',ascending=False)

,Neighborhood,Count
36,West Ridge,17
38,West Town,2
14,Heart of Chicago,2
28,Ravenswood Gardens,2
0,Armour Square,1
29,Ravenswood Manor,1
23,Mayfair,1
24,Merchant Park,1
25,Noble Square,1
26,Old Town,1


In [0]:
#List of all Indian Resturants in CHICAGO
Indian_Resturants_in_CHICAGO = Neighborhood_areas_with_Resturants.merge(data,how='inner',left_on='Neighborhood',right_on='Name') 

In [0]:
Indian_Resturants_in_CHICAGO.drop_duplicates(inplace=True)

In [66]:
Indian_Resturants_in_CHICAGO

,Neighborhood,Count,Name,Latitude,Longitude
0,Armour Square,1,Armour Square,41.840033,-87.633107
4,Avondale,1,Avondale,41.938921,-87.711168
5,Avondale Gardens,1,Avondale Gardens,41.938921,-87.711168
6,Belmont Central,1,Belmont Central,41.939796,-87.653328
7,Belmont Gardens,1,Belmont Gardens,41.939796,-87.653328
8,Belmont Terrace,1,Belmont Terrace,41.939796,-87.653328
9,Bowmanville,1,Bowmanville,41.972133,-87.693737
10,Bronzeville,1,Bronzeville,41.831272,-87.626548
11,East Hyde Park,1,East Hyde Park,41.794225,-87.592562
12,Edgewater,1,Edgewater,41.983369,-87.663952


In [0]:
#Neighborhoods in Chicago where there are no INDIAN resturants
Neighborhood_with_no_resturants = Neighborhood_with_no_resturants.merge(data,how='inner',left_on='Neighborhood',right_on='Name') 

In [0]:
Neighborhood_with_no_resturants.drop_duplicates(inplace=True)

#8. Plotting Neighborhoods of Chicago Having Resturants on MAP

In [0]:
chicago_map = folium.Map(location =[41.8781,-87.6298])
for name,lat,lng,count in zip(Indian_Resturants_in_CHICAGO.Name,Indian_Resturants_in_CHICAGO.Latitude,Indian_Resturants_in_CHICAGO.Longitude,Indian_Resturants_in_CHICAGO.Count):
  folium.CircleMarker(location=[lat,lng],radius=5,color='red',fill=True,fill_color='red').add_to(chicago_map)
  if name=='West Ridge':
    folium.Marker(location=[lat,lng],popup=folium.Popup((
      '<strong><font color=red>Neighborhood :</font> ' + name + '</strong><br>'+
      '<strong><font color=red>No of restutants  : </font>' + str(count) + '</strong><br>'),max_width=200),
      icon = folium.Icon(color='pink')).add_to(chicago_map)
  else:
    folium.Marker(location=[lat,lng],popup=folium.Popup((
      '<strong><font color=red>Neighborhood :</font> ' + name + '</strong><br>'+
      '<strong><font color=red>No of restutants  : </font>' + str(count) + '</strong><br>'),max_width=200)).add_to(chicago_map)


In [91]:
#chicago Map with all Indian restutants 
#Neighborhood with pink Marker is the Location where most INDIAN resturants can be found 
chicago_map

In [117]:
no_resturant_areas =  folium.Map(location = [41.8781,-87.6298])
for name,lat,lng in zip(Neighborhood_with_no_resturants.Name,Neighborhood_with_no_resturants.Latitude,Neighborhood_with_no_resturants.Longitude):
  folium.CircleMarker(location=[lat,lng],radius=5,color='red',fill=True,fill_color='red').add_to(no_resturant_areas)
  folium.Marker(location=[lat,lng],popup=folium.Popup((
      '<strong><font color=red>Neighborhood :</font> ' + name + '</strong><br>'),max_width=200),
      icon = folium.Icon(color='red')).add_to(no_resturant_areas)
no_resturant_areas

In [0]:
# important places for bussiness man to startup INDIAN RESTURANT
# These are imortant places where the population of INDIANS in high but there is no INDIAN resurant Nearby
# 1. Chicago Midway International Airport 
# 2. Little Italy
# 3. Washington Park
# 4. LilyDale


In [110]:
best_places_startup = pd.DataFrame(columns=['Name','Latitude','Longitude'])
best_places_startup

,Name,Latitude,Longitude


In [0]:
best_places_startup = best_places_startup.append(({'Name': 'Chicago Midway International Airport',
                                                  'Latitude':41.7786767,
                                                  'Longitude': -87.76920156251208}),ignore_index=True)

In [0]:
best_places_startup = best_places_startup.append(({'Name': 'Little Italy',
                                                  'Latitude':41.86448215,
                                                  'Longitude': -87.65597120016068}),ignore_index=True)

In [0]:
best_places_startup = best_places_startup.append(({'Name': 'Lilydale',
                                                  'Latitude':	41.7242009,
                                                  'Longitude': -87.625326}),ignore_index=True)

In [114]:
best_places_startup

,Name,Latitude,Longitude
0,Chicago Midway International Airport,41.778677,-87.769202
1,Little Italy,41.864482,-87.655971
2,Lilydale,41.724201,-87.625326


In [118]:
for name,lat,lng in zip(best_places_startup.Name,best_places_startup.Latitude,best_places_startup.Longitude):
  folium.Circle(location=[lat,lng],radius=5000,color='blue',fill_color='blue',opacity=0.5).add_to(no_resturant_areas)
no_resturant_areas

In [0]:
# The circle Mark shows where there are less INDINA Resturants and they are Most poupular tourist places
# Hence we can suggest these places to the person who whants to make wealth

# **Conclusion :**

1. Best Place in Chicago to find INDIAN Resturant : **West Ridge** (it has 17 INDIAN Restutants in its nearby)
2. Best Resturant in West Ridge to have Dinner : **TAHOORA** (it has got 7.8 ratings and 49 likes)
3. Best place for entrepreneur to start their bussiess : **Little Italy,Lilydale,Clearing** and it nearby